In [ ]:
ext=pd.read_csv('/kaggle/input/selection/extra_features.csv')
ext=ext.drop(['Unnamed: 0'],1)
ls=list(ext)
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv',usecols=['isFraud'])
ext=ext.loc[:590539]
ext['isFraud']=trn['isFraud']
import lightgbm as lgb
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
fnl=pd.DataFrame()
cols=[]
val=[]
trn=[]
for col in tqdm(ls):
    for i in range(50000,550000,50000):
        fit=ext.loc[0:i]
        pdct=ext.loc[i:i+40000]
        mod=lgb.LGBMClassifier(max_bins= 37,max_depth= 40,min_data_in_leaf= 110,min_gain_to_split= 1,num_leaves=48)
        mod.fit(fit[col].values.reshape(-1,1),fit['isFraud'])
        val.append(roc_auc_score(pdct['isFraud'],mod.predict_proba(pdct[col].values.reshape(-1,1))[:,1]))
        trn.append(roc_auc_score(fit['isFraud'],mod.predict_proba(fit[col].values.reshape(-1,1))[:,1]))
        cols.append(col)

In [ ]:
fnl['cols']=cols
fnl['validation']=val
fnl['train']=trn
fnl['validation']=fnl.groupby(['cols'])['validation'].transform('mean')
fnl['train']=fnl.groupby(['cols'])['train'].transform('mean')
fnl=fnl[['validation','train','cols']]
fnl=fnl.drop_duplicates(keep='last')

In [ ]:
fnl=fnl.sort_values(by='validation')
fnl.tail(100)['cols'].values